In [1]:
import os
import nltk
import re
import datetime
from functools import lru_cache
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import pickle
#from google.colab import files
import io
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from ray.tune.sklearn import TuneGridSearchCV
from ray.tune.sklearn import TuneSearchCV
from sklearn.linear_model import SGDClassifier
import ray

C:\Users\szymon\scoop\apps\python310\current\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ray.init()

2023-02-03 03:04:34,677	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.2.0


In [3]:
# remove any instance of <...> or numbers
REMOVAL_REGEX = re.compile(r"<.*?>|[0-9]")
SENTIMENT_POS = 1
SENTIMENT_NEG = 0
SENTIMENT_DICT = {0: "Negative", 1: "Positive"}

In [4]:
# Google Collab
#uploaded = files.upload()
#all_reviews_df = pd.read_csv(io.BytesIO(uploaded['IMDB Dataset.csv']))
# Local Files
# Not much point in reading the file if we use pickles anyway
all_reviews_df = pd.read_csv("./data/IMDB Dataset.csv")

# EDA

In [5]:
all_reviews_df.info()
all_reviews_df['sentiment'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


positive    25000
negative    25000
Name: sentiment, dtype: int64

In [6]:
encoder = preprocessing.LabelEncoder()
all_reviews_df["sentiment"] = encoder.fit_transform(all_reviews_df["sentiment"])
all_reviews_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
all_reviews_df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

# Pre-Processing

In [8]:
# Set up SpaCy
# You may need to run `python -m pip install 'spacy[transformers,lookups]'`
# You may need to run `python -m pip install spacy[cuda-autodetect]
# That is assuming you have a CUDA supported GPU (nVidia and AMD apparently support it)
# Otherwise it falls back to CPU.
# You can also do spacy.require_gpu(), however that will throw an error if it is not supported.
# If this returns False and you want it to be True, check if you installed `nvcc`, the Nvidia CUDA compiler.
# Also, you need to set the CUDA_PATH environment variable to where you installed the CUDA toolkit
# That may be done automatically if you install all components of the nVidia CUDA Toolkit though.
# Also, if this returns false, check if you did `python -m pip install cupy`
# (this will take a _while_) as you will be compiling some things with nvidia cuda compiler
# OR, as an alternative, I think you can do `python -m pip install cupy-wheel` OR
# `python -m pip installl cupy-cuda12x`
# This might be annoying depending on your GPU driver version.
# Actually, on an i7-6700k @4.7GHz, running it on the CPU took 1200 seconds to process 50k reviews
# With the help of the GPU, it actually took 1560 seconds. Somehow.
# On my laptop with an i7-6820HQ, it took double that time.
#print("Preferring GPU:",spacy.prefer_gpu())

# You need to run `python -m spacy download en_core_web_sm` or `python -m spacy download en_core_web_lg`
# Otherwise you get an error
# nlp = spacy.load("en_core_web_lg") # Potentially slower, but with more data.
nlp = spacy.load("en_core_web_sm")

In [9]:
preprocess_start_time = datetime.datetime.now()

In [10]:
# Add some stopwords that weren't caught after preprocessing
STOP_WORDS.add("s")
#STOP_WORDS.add("il")
#STOP_WORDS.add("ve")
#STOP_WORDS.add("ll")
#STOP_WORDS.add("t")
# Potentially crucial, but was not done initially.
# The pickled reviews were pre-processed without calling this, therefore some of the reviews
# Can be misclassified as they are missing these words.
STOP_WORDS.remove("not")
STOP_WORDS.remove("no")

@lru_cache(maxsize=51200)
def is_alpha_cached(token):
    return token.is_alpha

# Take in a row from a pandas dataframe
# And return a list of lemmatized words with no stop words, numbers, etc.

# We can't really create a cache here.
# As all reviews will be different.
# @lru_cache(maxsize=51200)
# I've noticed that preprocessing removes words like `not` which is not optimal.
# We could have removed `not` from STOP_WORDS.
def preprocess_review(review):
    review = REMOVAL_REGEX.sub(" ", review.lower())
    doc = nlp(review)
    # return " ".join(w.lemma_ for w in doc if (w.text not in STOP_WORDS and w.lemma_ not in STOP_WORDS) and is_alpha_cached(w))
    return " ".join(w.lemma_ for w in doc if ((w.text not in STOP_WORDS) and is_alpha_cached(w)))

In [11]:
# This step takes a **long** time (around 30-35 minutes on my machine.)
# Becasue of that, I have included the pickle file which can be used instead
PICKLE_POS = "./processed_positive_reviews.pickle"
PICKLE_NEG = "./processed_negative_reviews.pickle"
if os.path.isfile(PICKLE_POS):
    with open(PICKLE_POS, "rb") as pickle_file:
        print("Loading positive reviews from pickle")
        processed_positive_reviews = pickle.load(pickle_file)
else:
    print("Processing positive reviews from scratch")
    processed_positive_reviews = [preprocess_review(review) for _index, (review, sentiment) in all_reviews_df.iterrows() if sentiment == SENTIMENT_POS]
    with open(PICKLE_POS, "wb") as pickle_file:
        print("Writing positive reviews to pickle")
        pickle.dump(processed_positive_reviews, pickle_file)
    
if os.path.isfile(PICKLE_NEG):
    with open(PICKLE_NEG, "rb") as pickle_file:
        print("Loading negative reviews from pickle")
        processed_negative_reviews = pickle.load(pickle_file)
else:
    print("Processing negative reviews from scratch")
    processed_negative_reviews = [preprocess_review(review) for _index, (review, sentiment) in all_reviews_df.iterrows() if sentiment == SENTIMENT_NEG]
    with open(PICKLE_NEG, "wb") as pickle_file:
        print("Writing positive reviews to pickle")
        pickle.dump(processed_negative_reviews, pickle_file)

Loading positive reviews from pickle
Loading negative reviews from pickle


In [12]:
print("Sample processed positive review:\n", processed_positive_reviews[0], "\n----")
print("Sample processed negative review:\n", processed_negative_reviews[0], "\n----")

Sample processed positive review:
 reviewer mention watch oz episode hook right exactly happen thing strike oz brutality unflinche scene violence set right word trust faint hearted timid pull punch regard drug sex violence hardcore classic use word call oz nickname give oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inward privacy high agenda em city home aryan muslim gangsta latinos christians italian irish scuffle death stare dodgy dealing shady agreement far away main appeal fact go show dare forget pretty picture paint mainstream audience forget charm forget romance oz mess episode see strike nasty surreal ready watch develop taste oz get accustomed high level graphic violence violence injustice crooked guard sell nickel inmate kill order away mannered middle class inmate turn prison bitch lack street skill prison experience watch oz comfortable uncomfortable viewing that touch dark 
----
Sample processed negati

In [13]:
# We actually need to split the data 50/50
# To ensure that we have an even split of positive/negative reviews for training/testing
# We will split the data manually and then shuffle it.
SPLIT = 12500

random.shuffle(processed_positive_reviews)
random.shuffle(processed_negative_reviews)

reviews_train = processed_positive_reviews[:SPLIT] + processed_negative_reviews[:SPLIT]
sentiments_train = [SENTIMENT_POS] * SPLIT + [SENTIMENT_NEG] * SPLIT

reviews_test = processed_positive_reviews[SPLIT:25000] + processed_negative_reviews[SPLIT:25000]
sentiments_test = [SENTIMENT_POS] * SPLIT + [SENTIMENT_NEG] * SPLIT

# There is no need to shuffle it as train_test_split does it for us.

all_reviews = processed_positive_reviews + processed_negative_reviews
all_labels = [SENTIMENT_POS] * len(processed_positive_reviews) + [SENTIMENT_NEG] * len(processed_negative_reviews)

len(reviews_train), len(reviews_test), len(sentiments_train), len(sentiments_test)

(25000, 25000, 25000, 25000)

In [14]:
print("Sample train review:\n", reviews_train[0], "\nSentiment:", sentiments_train[0], "\n---")
print("Sample test review:\n", reviews_test[0], "\nSentiment:", sentiments_test[0], "\n---")

Sample train review:
 kid take movie love child age year give star emma roberts adorable title role expect generation robert future expose like britney spear lindsay lohan paris hilton refreshing girl look like work street enjoy see support cast include tate donovan rachel leigh cook barry bostwick monica parker cameo bruce willis final takeaway cute film note read book series comment base merit film 
Sentiment: 1 
---
Sample test review:
 james corbett autobiography roar crowd starting point lively remember fictionalize biography author heavyweight champion world succeed john sullivan turn century event narrative depict corbett brash likable intelligent young man conquest world boxing social prejudice time consider merely son irish immigrant lowly bank teller surprise take hour exciting amusing screen time prove compeer wrong bank teller film open wangle invitation sporting club fall love beautiful snobbish girl quarrel live home brawl clan corbett fight defeat club good professional 

In [15]:
print("Pre-processing the data took: ", (datetime.datetime.now() - preprocess_start_time).total_seconds())

Pre-processing the data took:  0.171816


# Vectorizers

## TF-IDF Vectorizer

In [16]:
PICKLE_TFIDF_VECT = "./tfidf_vectorizer.pickle"

if os.path.isfile(PICKLE_TFIDF_VECT):
    with open(PICKLE_TFIDF_VECT, "rb") as pickle_file:
        print("Loading TF-IDF Vectorizer from pickle")
        tfidf_vectorizer = pickle.load(pickle_file)
    tfidf_features = tfidf_vectorizer.transform(reviews_train)
else:
    print("Creating tf-idf vectorizer from scratch")
    # It turns out that tfidfVectorizer can actually preprocess our input
    # It might be easier to do that than using spacy
    # However I did not explore the potential performance difference
    # I wonder what the best value for this is
    tfidf_vectorizer = TfidfVectorizer(max_features=100)
    tfidf_features = tfidf_vectorizer.fit_transform(reviews_train)
    with open(PICKLE_TFIDF_VECT, "wb") as pickle_file:
        print("Writing TF-IDF Vectorizer to pickle")
        pickle.dump(tfidf_vectorizer, pickle_file)

Loading TF-IDF Vectorizer from pickle


## Unigram Vectorizer

In [17]:
PICKLE_UNIGRAM_VECT = "./unigram_vectorizer.pickle"

if os.path.isfile(PICKLE_UNIGRAM_VECT):
    with open(PICKLE_UNIGRAM_VECT, "rb") as pickle_file:
        print("Loading Unigram Vectorizer from pickle")
        unigram_vectorizer = pickle.load(pickle_file)
    unigram_features = unigram_vectorizer.transform(reviews_train)
else:
    print("Creating unigram vectorizer from scratch")
    unigram_vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=10000)
    unigram_features = unigram_vectorizer.fit_transform(reviews_train)
    with open(PICKLE_UNIGRAM_VECT, "wb") as pickle_file:
        print("Writing unigram vectorizer to pickle")
        pickle.dump(unigram_vectorizer, pickle_file)

Loading Unigram Vectorizer from pickle


## Bigram Vectorizer

In [18]:
PICKLE_BIGRAM_VECT = "./bigram_vectorizer.pickle"

if os.path.isfile(PICKLE_BIGRAM_VECT):
    with open(PICKLE_BIGRAM_VECT, "rb") as pickle_file:
        print("Loading Bigram Vectorizer from pickle")
        bigram_vectorizer = pickle.load(pickle_file)
    bigram_features = bigram_vectorizer.transform(reviews_train)
else:
    print("Creating bigram vectorizer from scratch")
    bigram_vectorizer = CountVectorizer(ngram_range=(2, 2), max_features=15000)
    bigram_features = bigram_vectorizer.fit_transform(reviews_train)
    with open(PICKLE_BIGRAM_VECT, "wb") as pickle_file:
        print("Writing bigram vectorizer to pickle")
        pickle.dump(bigram_vectorizer, pickle_file)

Loading Bigram Vectorizer from pickle


## Trigram Vectorizer

In [19]:
PICKLE_TRIGRAM_VECT = "./trigram_vectorizer.pickle"

if os.path.isfile(PICKLE_TRIGRAM_VECT):
    with open(PICKLE_TRIGRAM_VECT, "rb") as pickle_file:
        print("Loading Trigram Vectorizer from pickle")
        trigram_vectorizer = pickle.load(pickle_file)
    trigram_features = trigram_vectorizer.transform(reviews_train)
else:
    print("Creating trigram vectorizer from scratch")
    trigram_vectorizer = CountVectorizer(ngram_range=(3, 3), max_features=20000)
    trigram_features = trigram_vectorizer.fit_transform(reviews_train)
    with open(PICKLE_TRIGRAM_VECT, "wb") as pickle_file:
        print("Writing trigram vectorizer to pickle")
        pickle.dump(trigram_vectorizer, pickle_file)

Loading Trigram Vectorizer from pickle


# Creating The Models

In [20]:
sample_negative_review = "Horrible. I wanted the movie to end as quickly as possible. Waste of time and money, and the acting was not good either."
sample_positive_review = "This movie is great! I really, really loved it. Best movie I experienced in cinemas recently."

# .transform takes in a list of strings
# eg. ["This movie is banging!"]

print("Sample TF-IDF processed review:\n", tfidf_vectorizer.transform([sample_negative_review]), "\n----")
print("Sample Unigram processed review:\n", unigram_vectorizer.transform([sample_negative_review]), "\n----")
print("Sample Bigram processed review:\n", bigram_vectorizer.transform([sample_negative_review]), "\n----")
print("Sample Trigram processed review:\n", trigram_vectorizer.transform([sample_negative_review]), "\n----")

Sample TF-IDF processed review:
   (0, 87)	0.39841977055095673
  (0, 57)	0.2995195311612519
  (0, 31)	0.36286292688956506
  (0, 16)	0.521041357497608
  (0, 1)	0.5902509101672396 
----
Sample Unigram processed review:
   (0, 78)	1
  (0, 2924)	1
  (0, 3798)	1
  (0, 4294)	1
  (0, 5861)	1
  (0, 5935)	1
  (0, 6169)	1
  (0, 6838)	1
  (0, 7120)	1
  (0, 9072)	1
  (0, 9099)	1
  (0, 9729)	1 
----
Sample Bigram processed review:
  
----
Sample Trigram processed review:
  
----


In [21]:
# These steps can take a few minutes as well.
# In the future: Add these into pickles
tfidf_vectorized_test_reviews = tfidf_vectorizer.transform(reviews_test)

In [22]:
unigram_vectorized_test_reviews = unigram_vectorizer.transform(reviews_test)

In [23]:
bigram_vectorized_test_reviews = bigram_vectorizer.transform(reviews_test)

In [24]:
trigram_vectorized_test_reviews = trigram_vectorizer.transform(reviews_test)

## Hyper-Parameter Tuning

In [25]:
def display_hyperparams(cv_results):
    print("Best Parameters:", cv_results.best_params_)
    cv_mean_score = cv_results.cv_results_["mean_test_score"]
    cv_std_score = cv_results.cv_results_["std_test_score"]
    cv_params = cv_results.cv_results_["params"]
    for mean, std, params in zip(cv_mean_score, cv_std_score, cv_params):
        # The values have a lot of decimal places
        print(round(mean, 2), "\t+-\t", round(std, 2), "\tfor", params)

In [26]:
# Out of the 25k reviews for training, we can do hyperparameter tuning by taking let's say 20k for training, and 5k for testing

# I think this can be done in a better way
# What I do now is a little funky.

# total training reviews = 25k
# [12500 * positive] + [12500 * negative]
# What we want to do is:
# training = [10000 * positive] + [10000 * negative]
# testing = [2500 * positive] + [2500 * negative]
# Honestly, this should have been implemented in a better data structure than primitive lists.
# That's a "to-do" for future me.

# Keep in mind, the reviews are shuffled, but the order of positive and negative reviews are constant
kfold_reviews_train = reviews_train[:10000] + reviews_train[12500:22500]
kfold_sentiments_train = sentiments_train[:10000] + sentiments_train[12500:22500]

kfold_reviews_test = reviews_train[10000:12500] + reviews_train[22500:25000]
kfold_sentiments_test = sentiments_train[10000:12500] + sentiments_train[22500:25000]

print("Positive Train Reviews:", len([s for s in kfold_sentiments_train if s == SENTIMENT_POS]))
print("Negative Train Reviews:", len([s for s in kfold_sentiments_train if s == SENTIMENT_NEG]))
print("Positive Test Reviews:", len([s for s in kfold_sentiments_test if s == SENTIMENT_POS]))
print("Negative Test Reviews:", len([s for s in kfold_sentiments_test if s == SENTIMENT_NEG]))

len(kfold_reviews_train), len(kfold_reviews_test), len(kfold_sentiments_train), len(kfold_sentiments_test)

Positive Train Reviews: 10000
Negative Train Reviews: 10000
Positive Test Reviews: 2500
Negative Test Reviews: 2500


(20000, 5000, 20000, 5000)

### Naive Bayes
Naive Bayes does not have any hyper-parameters we can tune.

### Support Vector Machine

In [27]:
# Naive Bayes doesn't have any hyper-parameters (actually, it has `alpha`...)
# So we start off with SVM
hyper_svm = SGDClassifier()
hyper_svm_params = {
    #"epsilon" : [0.01]
    "alpha": [1e-7, 1e-4, 1e-1, 1],
    "epsilon": [0.01, 0.1, 0.25, 0.5, 1]
    # Used for svm.SVC() and not for SGDClassifier()
    #"kernel": ["linear", "rbf"],
    #"C": [0.1, 1, 2, 5, 10] # How harshly are wrong answers punished
}

# We can stick n_jobs = -1 here to ensure all CPU cores are used
# We can also run these on the GPU with the use_gpu=True flag
svm_cv = TuneGridSearchCV(
    hyper_svm, hyper_svm_params, early_stopping=True, max_iters=10, use_gpu=True
)
svm_cv.fit(tfidf_vectorizer.transform(kfold_reviews_train), kfold_sentiments_train)
display_hyperparams(svm_cv)
# The above svm_cv.best_params will be used in our models later on.

# Then see what results we get
# This should be in a function. Ah well.
svm_hyper_pred = svm_cv.predict(tfidf_vectorizer.transform(kfold_reviews_test))
print("Accuracy:", accuracy_score(kfold_sentiments_test, svm_hyper_pred))
print("Confusion Matrix:", "\n----\n", confusion_matrix(kfold_sentiments_test, svm_hyper_pred), "\n----")
print(classification_report(kfold_sentiments_test, svm_hyper_pred))

Best Parameters: {'alpha': 0.0001, 'epsilon': 0.1}
0.66 	+-	 0.01 	for {'alpha': 1e-07, 'epsilon': 0.01}
0.74 	+-	 0.01 	for {'alpha': 0.0001, 'epsilon': 0.01}
0.69 	+-	 0.01 	for {'alpha': 0.1, 'epsilon': 0.01}
0.62 	+-	 0.06 	for {'alpha': 1, 'epsilon': 0.01}
0.66 	+-	 0.02 	for {'alpha': 1e-07, 'epsilon': 0.1}
0.74 	+-	 0.01 	for {'alpha': 0.0001, 'epsilon': 0.1}
0.68 	+-	 0.03 	for {'alpha': 0.1, 'epsilon': 0.1}
0.51 	+-	 0.01 	for {'alpha': 1, 'epsilon': 0.1}
0.66 	+-	 0.03 	for {'alpha': 1e-07, 'epsilon': 0.25}
0.74 	+-	 0.01 	for {'alpha': 0.0001, 'epsilon': 0.25}
0.67 	+-	 0.02 	for {'alpha': 0.1, 'epsilon': 0.25}
0.67 	+-	 0.06 	for {'alpha': 1, 'epsilon': 0.25}
0.67 	+-	 0.01 	for {'alpha': 1e-07, 'epsilon': 0.5}
0.73 	+-	 0.0 	for {'alpha': 0.0001, 'epsilon': 0.5}
0.69 	+-	 0.03 	for {'alpha': 0.1, 'epsilon': 0.5}
0.57 	+-	 0.05 	for {'alpha': 1, 'epsilon': 0.5}
0.66 	+-	 0.01 	for {'alpha': 1e-07, 'epsilon': 1}
0.73 	+-	 0.01 	for {'alpha': 0.0001, 'epsilon': 1}
0.7 	+-	 0.

### K-Nearest Neighbours

In [28]:
# This can take a few minutes to complete.
# If you want, you can skip this. I ran it a few times and it consistently returned 19 as the best value.
# That's also mentioned later when running the final model.
hyper_knn = KNeighborsClassifier()
hyper_knn_params = {
    # I deleted 1, 2, 3 and 5 from this list
    # That is because I have tested it with 9, and it was better
    # Therefore we can reduce the computational complexity a little.
    "n_neighbors": [7, 9, 11, 13, 15, 17, 19]
}

knn_cv = TuneGridSearchCV(
    hyper_knn, hyper_knn_params, early_stopping=False, max_iters=1
)

knn_cv.fit(tfidf_vectorizer.transform(kfold_reviews_train), kfold_sentiments_train)
display_hyperparams(knn_cv)

knn_hyper_pred = knn_cv.predict(tfidf_vectorizer.transform(kfold_reviews_test))
print("Accuracy:", accuracy_score(kfold_sentiments_test, knn_hyper_pred))
print("Confusion Matrix:", "\n----\n", confusion_matrix(kfold_sentiments_test, knn_hyper_pred), "\n----")
print(classification_report(kfold_sentiments_test, knn_hyper_pred))

Best Parameters: {'n_neighbors': 19}
0.67 	+-	 0.01 	for {'n_neighbors': 7}
0.68 	+-	 0.01 	for {'n_neighbors': 9}
0.68 	+-	 0.01 	for {'n_neighbors': 11}
0.69 	+-	 0.01 	for {'n_neighbors': 13}
0.69 	+-	 0.01 	for {'n_neighbors': 15}
0.69 	+-	 0.01 	for {'n_neighbors': 17}
0.69 	+-	 0.01 	for {'n_neighbors': 19}
Accuracy: 0.6894
Confusion Matrix: 
----
 [[1604  896]
 [ 657 1843]] 
----
              precision    recall  f1-score   support

           0       0.71      0.64      0.67      2500
           1       0.67      0.74      0.70      2500

    accuracy                           0.69      5000
   macro avg       0.69      0.69      0.69      5000
weighted avg       0.69      0.69      0.69      5000



### N-Grams
As the N-Grams in this project utilise the same model as Naive Bayes, there are no hyper-parameters to tune.

## Final Models

### Naive Bayes

In [29]:
naive_bayes_model = MultinomialNB()
print("Training naive bayes model...")
naive_bayes_model.fit(tfidf_features, sentiments_train)

Training naive bayes model...


MultinomialNB()

#### Own Predictions

In [30]:
# You can run more manual predictions here
bayes_pred = naive_bayes_model.predict(tfidf_vectorizer.transform([sample_positive_review, sample_negative_review]))
for p in bayes_pred:
    print(SENTIMENT_DICT[p])

Positive
Negative


#### Testing Dataset

In [31]:
bayes_predictions = naive_bayes_model.predict(tfidf_vectorized_test_reviews)
print(accuracy_score(sentiments_test, bayes_predictions))
print(confusion_matrix(sentiments_test, bayes_predictions))
print(classification_report(sentiments_test, bayes_predictions))

0.73168
[[9001 3499]
 [3209 9291]]
              precision    recall  f1-score   support

           0       0.74      0.72      0.73     12500
           1       0.73      0.74      0.73     12500

    accuracy                           0.73     25000
   macro avg       0.73      0.73      0.73     25000
weighted avg       0.73      0.73      0.73     25000



### Support Vector Machine

In [32]:
# Let's get the best hyper-parameters automatically
# If this fails etc. A set of params that I got when running this notebook was:
# Best Parameters: {'alpha': 0.0001, 'epsilon': 1}
svm_model = SGDClassifier(epsilon=svm_cv.best_params_["epsilon"], alpha=svm_cv.best_params_["alpha"])
svm_model.fit(tfidf_features, sentiments_train)

SGDClassifier()

#### Own Predictions

In [33]:
svm_pred = svm_model.predict(tfidf_vectorizer.transform([sample_positive_review, sample_negative_review]))
for p in svm_pred:
    print(SENTIMENT_DICT[p])

Positive
Positive


#### Testing Dataset

In [34]:
svm_predictions = svm_model.predict(tfidf_vectorized_test_reviews)
print(accuracy_score(sentiments_test, svm_predictions))
print(confusion_matrix(sentiments_test, svm_predictions))
print(classification_report(sentiments_test, svm_predictions))

0.74116
[[9127 3373]
 [3098 9402]]
              precision    recall  f1-score   support

           0       0.75      0.73      0.74     12500
           1       0.74      0.75      0.74     12500

    accuracy                           0.74     25000
   macro avg       0.74      0.74      0.74     25000
weighted avg       0.74      0.74      0.74     25000



### K-Nearest Neighbours

In [35]:
# Let's get the param automatically after running hyper-parameter tuning
# If that fails, let's use 19 as that's the best value I got when I tested it
knn_model = KNeighborsClassifier(n_neighbors=knn_cv.best_params_["n_neighbors"])
knn_model.fit(tfidf_features, sentiments_train)

KNeighborsClassifier(n_neighbors=19)

#### Own Predictions

In [36]:
knn_pred = knn_model.predict(tfidf_vectorizer.transform([sample_positive_review, sample_negative_review]))
for p in knn_pred:
    print(SENTIMENT_DICT[p])

Positive
Negative


#### Testing Dataset

In [37]:
knn_predictions = knn_model.predict(tfidf_vectorized_test_reviews)
print(accuracy_score(sentiments_test, knn_predictions))
print(confusion_matrix(sentiments_test, knn_predictions))
print(classification_report(sentiments_test, knn_predictions))

0.69552
[[8026 4474]
 [3138 9362]]
              precision    recall  f1-score   support

           0       0.72      0.64      0.68     12500
           1       0.68      0.75      0.71     12500

    accuracy                           0.70     25000
   macro avg       0.70      0.70      0.69     25000
weighted avg       0.70      0.70      0.69     25000



### N-gram Model
Note to self: There is no actual unigram model. We use Naive Bayes for this.

#### Unigram Model

In [38]:
unigram_model = MultinomialNB()
unigram_model.fit(unigram_vectorized_test_reviews, sentiments_train)

MultinomialNB()

##### Own Predictions

In [39]:
unigram_pred = unigram_model.predict(unigram_vectorizer.transform([sample_positive_review, sample_negative_review]))
for p in unigram_pred:
    print(SENTIMENT_DICT[p])

Positive
Negative


##### Testing Dataset

In [40]:
unigram_predictions = unigram_model.predict(unigram_vectorized_test_reviews)
print(accuracy_score(sentiments_test, unigram_predictions))
print(confusion_matrix(sentiments_test, unigram_predictions))
print(classification_report(sentiments_test, unigram_predictions))

0.86728
[[11005  1495]
 [ 1823 10677]]
              precision    recall  f1-score   support

           0       0.86      0.88      0.87     12500
           1       0.88      0.85      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



#### Bigram Model

In [41]:
bigram_model = MultinomialNB()
bigram_model.fit(bigram_vectorized_test_reviews, sentiments_train)

MultinomialNB()

##### Own Predictions

In [42]:
bigram_pred = bigram_model.predict(bigram_vectorizer.transform([sample_positive_review, sample_negative_review]))
for p in bigram_pred:
    print(SENTIMENT_DICT[p])

Negative
Negative


##### Testing Dataset

In [43]:
bigram_predictions = bigram_model.predict(bigram_vectorized_test_reviews)
print(accuracy_score(sentiments_test, bigram_predictions))
print(confusion_matrix(sentiments_test, bigram_predictions))
print(classification_report(sentiments_test, bigram_predictions))

0.88256
[[10829  1671]
 [ 1265 11235]]
              precision    recall  f1-score   support

           0       0.90      0.87      0.88     12500
           1       0.87      0.90      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



#### Trigram Model

In [44]:
trigram_model = MultinomialNB()
trigram_model.fit(trigram_vectorized_test_reviews, sentiments_train)

MultinomialNB()

##### Own Predictions

In [45]:
trigram_pred = trigram_model.predict(trigram_vectorizer.transform([sample_positive_review, sample_negative_review]))
for p in trigram_pred:
    print(SENTIMENT_DICT[p])

Negative
Negative


##### Testing Dataset

In [46]:
trigram_predictions = trigram_model.predict(trigram_vectorized_test_reviews)
print(accuracy_score(sentiments_test, trigram_predictions))
print(confusion_matrix(sentiments_test, trigram_predictions))
print(classification_report(sentiments_test, trigram_predictions))

0.80264
[[10887  1613]
 [ 3321  9179]]
              precision    recall  f1-score   support

           0       0.77      0.87      0.82     12500
           1       0.85      0.73      0.79     12500

    accuracy                           0.80     25000
   macro avg       0.81      0.80      0.80     25000
weighted avg       0.81      0.80      0.80     25000

